In [1]:
#!pip install gradio langchain-google-genai langchain tensorflow pillow

In [14]:
# Core
import os
from dotenv import load_dotenv

# Image processing
import numpy as np
from PIL import Image

# TensorFlow model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  

# LLM: Gemini via Langchain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# UI
import gradio as gr


In [15]:
# 3. Load environment variables
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL = "gemini-2.0-flash"

In [16]:
# 4. Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY,
    model=GEMINI_MODEL,
    temperature=0.4
)

In [17]:
# Load the trained model (correct path based on your file)
cnn_model = load_model("saved_models/dementia_cnn_sequential_1_model_V5.keras")
print("CNN model loaded:", cnn_model.name)


CNN model loaded: sequential_1


In [18]:
# # 6. Class label map
# label_map = {
#     0: "Non-dementia",
#     1: "Very mild dementia",
#     2: "Mild dementia",
#     3: "Moderate dementia"
# }

In [19]:
label_map = ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented']

In [ ]:
def predict_from_mri(image):
    # 1. Convert to grayscale if not already
    if image.mode != "L":
        image = image.convert("L")

    # 2. Resize to match model input
    image = image.resize((224, 224))

    # 3. Convert to array
    img_array = img_to_array(image)  # shape: (224, 224, 1)

    # 4. Repeat the grayscale channel to create 3 channels (RGB format)
    img_array = np.repeat(img_array, 3, axis=-1)  # shape: (224, 224, 3)

    # 5. Normalize pixel values to [0, 1]
    img_array = img_array / 255.0

    # 6. Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)  # shape: (1, 224, 224, 3)

    # ✅ 7. Predict using the model
    preds = cnn_model.predict(img_array, verbose=0)

    # 8. Get prediction with confidence
    predicted_class = np.argmax(preds)
    confidence = float(preds[0][predicted_class])
    label = label_map[predicted_class]

    # Optional: Print prediction details
    print(f"Model prediction: {label} ({confidence:.2f} confidence)")
    print("Class probabilities:")
    for i, prob in enumerate(preds[0]):
        print(f"  {label_map[i]}: {prob:.2f}")

    return label, confidence, preds[0]


NameError: name 'img_array' is not defined

In [ ]:
# 8. Function: Use Gemini to generate natural language explanation
def generate_gemini_summary(input_text):
    prompt = PromptTemplate.from_template("""
You are a dementia diagnostic assistant.

Interpret the following patient data or classification and explain what it means in simple, compassionate terms for a family or clinical audience:

"{input_text}"

Do not diagnose or suggest treatment. Just explain what it may indicate about dementia stages.
""")
    final_prompt = prompt.format(input_text=input_text)
    response = llm.invoke(final_prompt)
    return response.content.strip()

In [ ]:
# 9. Combined chatbot function
def multimodal_chatbot(mri_image, symptom_text):
    responses = []

    if mri_image:
        diagnosis, confidence, all_probs = predict_from_mri(mri_image)
        
        # Format probabilities for all classes
        class_probs = [f"{label_map[i]}: {prob:.2f}" for i, prob in enumerate(all_probs)]
        prob_text = ", ".join(class_probs)
        
        responses.append(f"**MRI Classification:** {diagnosis} (Confidence: {confidence:.2f})")
        responses.append(f"**All Probabilities:** {prob_text}")
        
        explanation = generate_gemini_summary(f"MRI indicates: {diagnosis} with {confidence:.2f} confidence")
        responses.append(f"**Gemini Summary for MRI:**\n{explanation}")

    if symptom_text:
        explanation = generate_gemini_summary(f"Symptoms: {symptom_text}")
        responses.append(f"**Gemini Summary for Symptoms:**\n{explanation}")

    if not responses:
        return "Please upload an MRI image or enter symptom information."

    return "\n\n".join(responses)

In [ ]:
# Model validation test
def validate_model(test_image_path):
    """Test model on a single known image"""
    test_image = Image.open(test_image_path)
    label, confidence, probs = predict_from_mri(test_image)
    print(f"Predicted: {label}, Confidence: {confidence:.4f}")
    print(f"All probabilities: {probs}")
    return label, confidence, probs

In [ ]:
# 10. Gradio UI
gr.Interface(
    fn=multimodal_chatbot,
    inputs=[
        gr.Image(type="pil", label="Upload MRI Image"),
        gr.Textbox(lines=4, placeholder="Describe symptoms here...", label="Symptom Description")
    ],
    outputs="markdown",
    title="Dementia MRI & Symptom Assistant (Powered by Gemini)",
    description="Upload an MRI or enter symptoms. The assistant will classify dementia and explain the findings using Google's Gemini 2.0 model."
).launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


2025-05-12 19:41:20.878085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
E0000 00:00:1747100480.881737  440785 meta_optimizer.cc:967] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.
